In [34]:
from dotenv import load_dotenv
load_dotenv()
import logging
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from utils.fetch_docs import fetch_documents
from langchain_google_genai import ChatGoogleGenerativeAI
from code_reflection_agent import final_agent
from langgraph.types import Command, interrupt
from typing import  Literal, TypedDict





# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)

logger = logging.getLogger(__name__)


template = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent 
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify, you can suggest and see if user confirms the suggestions.

After you are able to discern all the information, call the tool AgentInstruction"""

In [35]:
class AgentInstructions(BaseModel):
    """Instructions on how to build the Agent"""
    objective: str = Field(description= "What is the primary objective of the agent")
    usecases: List[str] = Field(description= "What are the various responsibilities of the agent which it needs to fulfill")
    examples : str = Field(description= "What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class AgentBuilderState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    json_code: str = Field("The json code generated")
    python_code: str = Field("The Python code generated")

class ArchitectureEvaluationState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    url: str = Field("url of the agent architecture to evaluate against")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0)

def requirement_analysis_node(state: AgentBuilderState) -> Command[Literal["requirement_analysis", "code_node"]]:
    llm_with_tool = llm.bind_tools([AgentInstructions])
    response = llm_with_tool.invoke([SystemMessage(content=template)] + state["messages"])
    
      # Construct the final answer from the arguments of the last tool call  
    if len(response.tool_calls) == 0: # info is needed
        value = interrupt(response.content)
        return Command(goto="requirement_analysis_node", update= {"messages": [response, HumanMessage(content=value)]} )
        
    agent_instructions = response.tool_calls[0]
    agent_instructions = AgentInstructions(**agent_instructions["args"])
    return Command(goto="code_node", update= {"messages": [response], "agent_instructions": agent_instructions } )



In [36]:
CODE_GEN_PROMPT = PromptTemplate.from_template( """
You are an expert Python programmer specializing in AI agent development via the Langgraph and Langchain SDK . Your primary task is to generate compilable, logical, and complete Python code for a LangGraph state graph based on user input below. You must prioritize LLM-based implementations for relevant tasks and consider advanced graph architectures.

**Input:**
<INPUT>
<OBJECTIVE>
{objective}
</OBJECTIVE>
<USECASES>
{usecases}
</USECASES>
<EXAMPLES>
{examples}
</EXAMPLES>
</INPUT>
---
**Phase 1: Evaluating best architecture for the given INPUT**

1.  **Identify Potential Architectures:** Consider if the described INPUT aligns with or would benefit from known advanced LangGraph architectures such as:
    * **Plan and Execute**: Does the INPUT imply an agent which might need a planning step (e.g., breaking down a complex task) followed by the execution of those plans by one or more action nodes?
    * **Agent Supervisor / Hierarchical Agent Teams**: Is the INPUT best served by a supervisor agent dispatching tasks to specialized worker agents, or a hierarchy of agents making decisions and delegating?
    * **Multi-Agent Collaboration (e.g., Swarm Architecture)**: Does the problem benefit from multiple agents working in parallel or collaboratively, perhaps sharing insights or contributing to a common goal?
    * **Reflection / Self-Correction (e.g., Self-Discover frameworks)**: Are there indications of iterative refinement, where results are evaluated and the process is adjusted?
    * **Human in the Loop (HITL)**: Does the `description` of any node, or the overall process, imply a need for human review, approval, correction, or explicit input at specific stages (e.g., before executing a critical action, when confidence is low, or for subjective assessments)?

2.  **Architectural Decision:**
    * If you determine that one or more of these architectures are strongly applicable to the INPUT, choose to implement it.
    * If no specific advanced architecture seems directly applicable for the given INPUT, proceed with a standard stateful graph construction based on the explicit langgraph nodes and edges.

3.  **Initial Comment:** At the very beginning of your generated Python script, include a comment block stating:
    * Which LangGraph architecture(s) (if any) you've identified and chosen to implement, with a brief justification based on your interpretation of the INPUT, provide dry runs of the usecases/examples.
    * If you are proceeding with a standard graph, mention that.

4. Generate a JSON representation of the architecture you have chosen, including:
a.  `nodes`: A dictionary where each key is a unique node ID. The value for each node ID is an object containing:
    * `id`: The node's identifier.
    * `schema_info`: A string describing the structure of the `GraphState` (e.g., "GraphState:\n type: TypedDict\n fields:\n - name: input\n type: str..."). You will need to parse this to define the `GraphState` TypedDict.
    * `input_schema`: The expected input schema for the node (typically "GraphState").
    * `output_schema`: The schema of the output produced by the node (typically "GraphState", indicating a partial update).
    * `description`: A natural language description of what the node does. This is crucial for determining implementation strategy and overall architecture.
    * `function_name`: The suggested Python function name for this node.
    * `code` (optional): A string containing Python code for the node's function. **Treat this `code` primarily as an illustration or a very basic version. Prioritize LLM-based solutions if the `description` suggests a more robust approach is needed.**

b.  `edges`: A list of objects, each describing a directed edge in the graph. Each edge object contains:
    * `source`: The ID of the source node (or "__START__" for the graph's entry point).
    * `target`: The ID of the target node (or "__END__" for a graph termination point).
    * `routing_conditions`: A natural language description of the condition under which this edge is taken, especially for conditional edges.
    * `conditional`: A boolean flag, `true` if the edge is part of a conditional branch, `false` otherwise.


---

** Phase 2: Graph Creation
**Phase 2: Python Code Generation**

Generate a single, self-contained, and compilable Python script that implements your chosen strategy.

1.  **Imports:** Include all necessary Python libraries (e.g., `typing`, `langgraph.graph`, `langgraph.checkpoint.memory`, LLM client libraries like `langchain_openai`, `langchain_google_genai`, `langchain_core.pydantic_v1`, `langchain_core.tools`, `re`).

2.  **State Definition (`GraphState`):**
    *  Define a `GraphState` class using `MessagesState` (langgraph prebuilt class).

3.  **Node Implementation (Python Functions):**
    For each conceptual node in your chosen architecture (these may map directly to JSON you define):
    * Create a Python function. This function must accept the `GraphState` and return a dictionary representing the partial update to the state.
    * **Decision Logic for Implementation (Prioritize LLM, No Mock Data):**
        * **Default to LLM-Based Solutions:** Your default stance should be to implement an **LLM-based solution** if the node's `description` (from JSON or your architectural design) suggests tasks like:
            * Natural Language Understanding (NLU)
            * Complex classification or routing
            * Content generation or summarization
            * Tool selection and usage
            * Planning or complex decision-making.
            * Any task where an LLM would provide more robust, flexible, or intelligent behavior than simple hardcoded logic.
        * **Handling Provided `code`:** If `code` is present in the JSON for a node, treat it as a **low-priority hint or a simplistic example**. Do **not** simply copy it if an LLM approach is more appropriate for the described task.
        * **Algorithmic Logic (Use Sparingly):** Only use purely algorithmic Python code (like from the `code` attribute or written new) if the node's task is genuinely simple, deterministic (e.g., basic data formatting, fixed calculation), *and* an LLM would offer no significant benefit for that specific, narrow function.
        * **Functional LLM Calls:** When an LLM is used, instantiate a generic model (e.g., `llm = ChatOpenAI(model="gpt-3.5-turbo")` or `llm = ChatGoogleGenerativeAI(model="gemini-pro")`) and include a **functional, descriptive prompt** relevant to the node's task. Ensure the code for the LLM call is complete and not just a comment. Add a `TODO` comment for the user to specify API keys and potentially refine the model/prompt.
        * **No Mock Data:** Generated functions must be logical and aim for completeness. **Avoid using mock data or overly simplistic placeholder logic** where an LLM or a proper algorithmic implementation is expected.
        * **Structured Output & Tools:** If the task implies structured output from an LLM or the use of tools, define necessary Pydantic models and/or LangChain tools, and integrate them with the LLM call.
            * Define a Pydantic model (e.g., `from langchain_core.pydantic_v1 import BaseModel, Field`) representing the desired structured output.
            * If implementing an LLM call, configure it to use the Pydantic model for its output (e.g., with OpenAI's function calling/tool usage features, or by instructing the LLM to generate JSON conforming to the model).
        * **Tool Definition and Usage:** If a node's `description` (or your architectural design) implies the LLM within that node needs to interact with external systems, perform specific actions, or fetch data (e.g., "search customer database," "get weather update"):
                * Define these capabilities as discrete LangChain tools using the `@tool` decorator (e.g., `from langchain_core.tools import tool`).
                * **Crucially, each tool's internal Python function should be self-contained and directly perform its advertised action** (e.g., make a specific API call to an external service, run a local script, perform a calculation, retrieve data algorithmically). **Avoid embedding a *new, separate general-purpose LLM call within the tool's own implementation logic* unless the tool's explicit and documented purpose is to be a specialized, self-contained sub-agent (which is an advanced case).** The primary LLM within the graph node is responsible for *deciding to call* the tool and for interpreting its output.
                * Bind these well-defined tools to the LLM instance operating within that graph node. The node's LLM will then intelligently decide when to call a tool and with what inputs.
        * **Human in the Loop Nodes:** If you've designed a HITL step as a dedicated node, its function might primarily format data for human review and then process the subsequent human input (which would be added to the state, potentially by an external mechanism or a subsequent node). The graph might pause using an interruption mechanism tied to this node.
        * **State Coherence:** Ensure variable assignments and updates within node functions are coherent with the `GraphState` definition and how state is managed in LangGraph.

4.  **Graph Construction (`StatefulGraph`):**
    * Instantiate `StatefulGraph(GraphState)`.
    * Add each implemented node function to the graph using `graph.add_node("node_id", node_function)`.
    * Set the graph's entry point using `graph.add_edge(START, "entry_node_id")` where `"entry_node_id"` is the target of the edge originating from `"__START__"`.

5.  **Edge Implementation:**
    * Iterate through the `edges` list in the JSON.
    * **Regular Edges:** If `conditional` is `false`:
        * If `target` is `__END__`, use `graph.add_edge(source_node_id, END)`.
        * Otherwise, use `graph.add_edge(source_node_id, target_node_id)`.
    * **Conditional Edges:** If `conditional` is `true`:
        * The `source` node of these conditional edges is expected to produce some output in the `GraphState` (e.g., an `intent` field) that determines the next path.
        * Create a separate routing function (e.g., `def route_after_source_node(state: GraphState) -> str:`).
        * This routing function must inspect the relevant fields in the `state` and return the string ID of the next node to execute, based on the logic described in the `routing_conditions` for each conditional edge originating from that source.
        * Use `graph.add_conditional_edges(source_node_id, routing_function, {{ "target_id_1": "target_id_1", "target_id_2": "target_id_2", ... "__END__": END }})`. The keys in the dictionary are the possible return values from your routing function, and the values are the actual node IDs or `END`.

6.  **Compilation:**
    * Instantiate an `InMemoryCheckpointer`: `checkpointer = InMemoryCheckpointer()`.
    * Compile the graph: `final_app = graph.compile(checkpointer=checkpointer)`. The compiled graph must be assigned to a variable named `final_app`.

---
**Phase 3: Required Keys/Credentials Identification**

After generating the complete Python script, add a separate section at the end of your response, clearly titled:
`## Required Keys and Credentials`

In this section, list all environment variables or API keys a user would need to set for the generated code to execute successfully (e.g., `OPENAI_API_KEY`, `GOOGLE_API_KEY`, tool-specific keys). If no external keys are needed, state that.

---
**Important Considerations (General):**
* The primary goal is **compilable, logical, and functionally plausible Python code** that intelligently interprets the JSON input.
* Focus on creating a system that leverages LLMs effectively for tasks suited to them.
* Ensure node functions correctly update and return relevant parts of the `GraphState`.
* If the provided `code` in the JSON uses specific libraries (e.g., `re`), make sure the corresponding import is included at the top of the script.
* Handle `__START__` and `__END__` correctly in edge definitions. `langgraph.graph.START` and `langgraph.graph.END` should be used.

Please generate the Python code and the list of required keys now:
""")

def code_node(state: AgentBuilderState):
    """Produce the final python code"""
    instructions: AgentInstructions = state["agent_instructions"]
    code_ouptut = llm.invoke([HumanMessage(content=CODE_GEN_PROMPT.format(
        objective = instructions.objective, usecases = instructions.usecases, examples = instructions.examples
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated final python code!")],
        "python_code": code_ouptut.content,
    }


In [37]:
checkpointer = MemorySaver()
workflow = StateGraph(AgentBuilderState)
workflow.add_node("code_node",code_node)
workflow.add_node("requirement_analysis", requirement_analysis_node)

workflow.add_edge("code_node", END)
workflow.add_edge(START, "requirement_analysis")
graph = workflow.compile(checkpointer=checkpointer)

In [38]:
query= " need to create a worklow with the objective of managing my social media.  It should be able to tell me trends from social media for sports, get me all the relevant people I should contant for a spsonsoring a specific post,  suggest me content I should be posting, make content for me if I give it a description * Identifying social media trends in sports. *   Finding relevant people for sponsoring specific posts. *   Suggesting content to post. *   Generating content based on a description.  Examples: Given I ask it about trends in football, it goes through recent viral reels in football and tell me Q&A reels are trending If I tell it I want people who would be interested in a post about UCL football, it gives me current players like Dembele etc If I tell it I want to make a post about UCL football, it goes through what is trending and an agent that thinks about social media posts and tells me we should post a highlight reel"

In [39]:
# Run the graph until the interrupt is hit.
config = {"configurable": {"thread_id": "some_id"}}
result = graph.invoke({"messages": HumanMessage(content=query)}, config=config) 




In [20]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

 need to create a worklow with the objective of managing my social media.  It should be able to tell me trends from social media for sports, get me all the relevant people I should contant for a spsonsoring a specific post,  suggest me content I should be posting, make content for me if I give it a description * Identifying social media trends in sports. *   Finding relevant people for sponsoring specific posts. *   Suggesting content to post. *   Generating content based on a description.  Examples: Given I ask it about trends in football, it goes through recent viral reels in football and tell me Q&A reels are trending If I tell it I want people who would be interested in a post about UCL football, it gives me current players like Dembele etc If I tell it I want to make a post about UCL football, it goes through what is trending and an agent that thinks about social media posts and tells me we should po

In [22]:
print(result["python_code"])

```python
import operator
from typing import TypedDict, Annotated, List, Optional
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
import json

# --- Phase 1: Evaluating best architecture for the given INPUT ---

"""
LangGraph Architecture Chosen: Hierarchical Agent Team with Supervisor and Specialized Worker Agents.

Justification:
The user's objective of "managing social media" with distinct use cases like "Identifying social media trends", "Finding relevant people for sponsoring", "Suggesting content", and "Generating content" strongly suggests a need for specialized expertise. A single, monolithic agent would struggle to efficiently handle such diverse tasks.

1.  **Supervisor Agent**: A central 'supervisor' node is ideal for in

In [27]:
response = llm.invoke([HumanMessage(content="implement basic tool calling via langgraph and langchain, use ToolNode and tools_condition")])

response.pretty_print()

================================== Ai Message ==================================

This example demonstrates how to implement basic tool calling using `langgraph` with `ToolNode` and `tools_condition`.

**Key Concepts:**

1.  **`StateGraph`**: Defines the state and nodes of our graph.
2.  **`Annotated` and `add_messages`**: Used to manage the conversation history in the graph's state.
3.  **LangChain Tools**: Standard LangChain tools defined using the `@tool` decorator.
4.  **`ChatOpenAI` with `bind_tools`**: An LLM capable of understanding and generating tool calls.
5.  **`ToolNode`**: A built-in LangGraph node that executes tool calls present in the state.
6.  **`tools_condition`**: A helper function from `langgraph.prebuilt` that checks if the last message from the LLM contains tool calls. This is crucial for routing.
7.  **Graph Flow**:
    *   Start at the `llm_node`.
    *   After the `llm_node`, use `tools_condition` to decide:
        *   If tool calls are present, go to `tool_n

In [ ]:
print(graph.invoke(Command(resume="Edited text"), config=config)) 